In [1]:
#ThermostatSC2

In [2]:
import datetime
import time
import random
import logging
import uuid

from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN_POST = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

GRAPH_PATTERN_REACT = """?setting rdf:type saref:SetLevelCommand .
                    ?setting saref:hasValue ?desired_temp .
                    ?setting saref:isMeasuredIn saref:TemperatureUnit .
                    ?setting saref:hasTimestamp ?timestamp .
                    ?setting saref:isCommandOf ?room_id .
                    ?setting saref:relatesToProperty saref:Temperature .
                    ?setting saref:commandIssuedBy ?device_id .
                    """

In [4]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [5]:
def handle_react_change_desired_temp(bindings):
    for binding in bindings:
        print(f'React binding: {binding}')
        #change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_temperature"
            + f'?desired_temperature={int(binding["desired_temp"])}'
        )
    return []

In [6]:
import threading

def start_sensor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)
    
    post_ki_id = register_post_knowledge_interaction(
        GRAPH_PATTERN_POST,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )
    
    react_ki_id = register_react_knowledge_interaction(
        GRAPH_PATTERN_REACT,
        None,
        "set-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )
    
    def measurement_loop(post_ki_id, kb_id, ke_endpoint):
        while True:
            value = get_temp_now()

            post(
                [
                    {
                        "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                        "temp": f"{get_temp_now()}",
                        "timestamp": f'"{get_timestamp_now()}"',  # ISO 8601 format
                        "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                        "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                    }
                ],
                post_ki_id,
                kb_id,
                ke_endpoint,
            )
            logger.info(f"published measurement of {value} units at {get_timestamp_now()}")

            time.sleep(5)

    # Start the measurement_loop function in a separate thread
    measurement_thread = threading.Thread(target=measurement_loop, args=(
        post_ki_id,
        kb_id,
        ke_endpoint,
    ))
    measurement_thread.start()
    
    # Start the start_handle_loop function in a separate thread
    start_handle_loop(
        {
            react_ki_id: handle_react_change_desired_temp,
        },
        kb_id,
        ke_endpoint,
    )
    

In [7]:
start_sensor_kb(
    "http://example.org/sensor",
    "Thermostat1_temp",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered Thermostat1_temp
INFO:utils:received issued knowledge interaction id: http://example.org/sensor/interaction/post-measurements
INFO:utils:received issued knowledge interaction id: http://example.org/sensor/interaction/set-desired-temp
INFO:__main__:published measurement of -2 units at 2023-06-19T19:23:28+00:00
INFO:__main__:published measurement of 3 units at 2023-06-19T19:23:33+00:00
INFO:__main__:published measurement of 1 units at 2023-06-19T19:23:38+00:00


React binding: {'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"', 'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"', 'desired_temp': '21', 'timestamp': '"2023-06-19T19:23:40+00:00"', 'setting': '<http://0.0.0.0:8001/thermostat/settings/7173ee51-8efd-4ffe-9dfc-c651de03cd0c>'}


INFO:__main__:published measurement of 19 units at 2023-06-19T19:23:44+00:00
INFO:__main__:published measurement of 19 units at 2023-06-19T19:23:49+00:00
INFO:__main__:published measurement of 23 units at 2023-06-19T19:23:54+00:00


React binding: {'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"', 'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"', 'desired_temp': '36', 'timestamp': '"2023-06-19T19:23:54+00:00"', 'setting': '<http://0.0.0.0:8001/thermostat/settings/c817fb09-8ea1-4100-a662-a62c8c892bea>'}


INFO:__main__:published measurement of 39 units at 2023-06-19T19:23:59+00:00
INFO:__main__:published measurement of 37 units at 2023-06-19T19:24:04+00:00


React binding: {'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"', 'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"', 'desired_temp': '5', 'timestamp': '"2023-06-19T19:24:07+00:00"', 'setting': '<http://0.0.0.0:8001/thermostat/settings/f26f8341-0198-4f12-86a3-1bd98b1a5fb1>'}


INFO:__main__:published measurement of 5 units at 2023-06-19T19:24:09+00:00
INFO:__main__:published measurement of 7 units at 2023-06-19T19:24:14+00:00
INFO:__main__:published measurement of 8 units at 2023-06-19T19:24:20+00:00
INFO:__main__:published measurement of 4 units at 2023-06-19T19:24:25+00:00
INFO:__main__:published measurement of 8 units at 2023-06-19T19:24:30+00:00
INFO:__main__:published measurement of 2 units at 2023-06-19T19:24:35+00:00
INFO:__main__:published measurement of 8 units at 2023-06-19T19:24:40+00:00
INFO:__main__:published measurement of 6 units at 2023-06-19T19:24:45+00:00
INFO:__main__:published measurement of 3 units at 2023-06-19T19:24:50+00:00
INFO:__main__:published measurement of 5 units at 2023-06-19T19:24:55+00:00
INFO:__main__:published measurement of 7 units at 2023-06-19T19:25:00+00:00
INFO:__main__:published measurement of 8 units at 2023-06-19T19:25:05+00:00
INFO:__main__:published measurement of 3 units at 2023-06-19T19:25:10+00:00
INFO:__main_

KeyboardInterrupt: 